# GIQ Benchmark: Wild Images Pipeline

This notebook runs the full pipeline (Feature Extraction -> Training -> Evaluation) exclusively on **wild images** for the GIQ Benchmark.

**Prerequisites:**
- Attach the GIQ dataset to your Kaggle notebook.
- Enable the GPU (T4 x2 or P100) in the Kaggle notebook settings.
- Ensure your Kaggle notebook has Internet access to clone the repository and download models.

## 1. Setup Environment & Repository
Clone the GitHub repository and install required dependencies.

In [ ]:
# Replace with your actual repository URL (e.g., https://github.com/your-username/geoprobe.git)
REPO_URL = "https://github.com/your-username/your-repo-name.git"

!git clone $REPO_URL geoprobe
%cd geoprobe

# Install dependencies (Kaggle already has PyTorch/torchvision)
!pip install transformers timm open3d mitsuba ruff

## 2. Link Kaggle Dataset
Link the dataset attached to the notebook into the expected `data/giq` directory.

In [ ]:
import os

# Update this path to match your attached Kaggle dataset directory name
KAGGLE_DATASET_PATH = "/kaggle/input/giq-dataset" 

os.makedirs("data/giq", exist_ok=True)
if not os.path.exists("data/giq/renderings"):
    os.symlink(f"{KAGGLE_DATASET_PATH}/renderings", "data/giq/renderings")
if not os.path.exists("data/giq/splits"):
    os.symlink(f"{KAGGLE_DATASET_PATH}/splits", "data/giq/splits")

print("Dataset symlinks created successfully!")
!ls -la data/giq

## 3. Extract Features for Wild Images
Extract features for the train, val, and test splits using a pre-trained backbone (e.g., `siglip2`).

In [ ]:
%%bash
BACKBONE="siglip2"

for SPLIT in train val test; do
    echo "=============================================="
    echo "Extracting wild features for $SPLIT split..."
    echo "=============================================="
    python scripts/extract_wild_features.py \n        --model $BACKBONE \n        --split $SPLIT \n        --batch_size 32 \n        --device cuda
done

## 4. Train Probes on Wild Data
Train linear probes on the extracted wild features for both Mental Rotation and Symmetry tasks.

In [ ]:
%%bash
BACKBONE="siglip2"

echo "=============================================="
echo "Training Mental Rotation probe (wild-wild)..."
echo "=============================================="
python scripts/train_probe.py \n    --task rotation \n    --backbone $BACKBONE \n    --probe linear \n    --image_type wild-wild \n    --epochs 10 \n    --batch_size 32

echo "=============================================="
echo "Training Symmetry probe (wild)..."
echo "=============================================="
python scripts/train_probe.py \n    --task symmetry \n    --backbone $BACKBONE \n    --probe linear \n    --image_type wild \n    --epochs 10 \n    --batch_size 32

## 5. Evaluate Probes
Evaluate the trained probes on the test split and save the results.

In [ ]:
%%bash
BACKBONE="siglip2"

echo "=============================================="
echo "Evaluating Mental Rotation..."
echo "=============================================="
python scripts/evaluate.py \n    --task rotation \n    --backbone $BACKBONE \n    --probe linear \n    --image_type wild-wild

echo "=============================================="
echo "Evaluating Symmetry..."
echo "=============================================="
python scripts/evaluate.py \n    --task symmetry \n    --backbone $BACKBONE \n    --probe linear \n    --image_type wild

## 6. Package Outputs
Compress the extracted features, trained models, and result metrics into a single archive so you can easily download it from Kaggle"s Output section.

In [ ]:
%%bash
echo "Packaging outputs..."
tar -czvf /kaggle/working/giq_wild_outputs.tar.gz data/giq/features experiments/
echo "Done! You can download giq_wild_outputs.tar.gz from the right panel -> Data -> Output."